<a href="https://colab.research.google.com/github/JF11579/Plotly_RE/blob/main/SameHouse/SP500_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [79]:
#!pip install pandas
#!pip3 install --upgrade pip
#!pip install plotly
#!pip install scikit-learn




In [80]:
import pandas as pd
from google.colab import drive  # <<<<<<<<<------
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


# Options

In [81]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data

In [82]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Real_Estate_Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DATA/Real_Estate_Data


In [83]:
houses = pd.read_csv('Combined_data_Jan_10_3.csv')

<ipython-input-83-46007e9c02e9>:1: DtypeWarning:

Columns (5,17,19,21,22,23,24,26,28,29,32,33,35,36,37,41,43,45,46,47,48,49,51,54,55,56,57,59,60,62,72,73,74,76,85,86,89,90,104,107,109,111,113,115,117,123,124,125,127,135,143,144,151,152,154,156,158,160,161,164,167,174,180,181,184,185,187,188,189,190,191,192,193,194,199,200,202,203,208,214,216,217,220) have mixed types. Specify dtype option on import or set low_memory=False.



In [84]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

In [85]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Uncatergorized_Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DATA/Uncatergorized_Data


In [86]:
SP500 = pd.read_csv('S&P500_yahoo.csv')

# Pre-process

## Pre-process houses

In [87]:
data = houses
data = data.dropna(subset=["Sold_Price"]).query("`Sold_Price` != 0")
data = data[['Address', 'Town','Zip','State','Style',
                    'ML#','Sold_Price','Baths_Full', 'Baths_Half',
                    'Bath','Bedrooms',
                      'Appearance','REO' , 'School_District_Name',
                 'Approx_Int_Square_Footage','Street_Name','Contract_Date', 'Lot_Square_Footage', 'Lot_Size']]


## convert Date

In [88]:
# Convert 'Contract_Date' to datetime, coerce errors to NaT
data['Contract_Date'] = pd.to_datetime(data['Contract_Date'], errors='coerce')

# Extract the year
data['Year'] = data['Contract_Date'].dt.year

# Drop rows where 'Year' is NaN (including where 'Contract_Date' conversion failed)
data = data.dropna(subset=['Year'])

data['Year'] = data['Year'].astype(int)

#
data["Contract_Date"] = data["Contract_Date"].dt.strftime("%Y-%m-%d")

data["Month_Year"] = data["Contract_Date"].str[:7]


<ipython-input-88-b2f84e0ba9e0>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-88-b2f84e0ba9e0>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [89]:
data.head(2)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11


In [90]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

## Create Medain Column

In [91]:
#data = houses
median_sold_price_all = data.groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_price_all.rename(columns={'Sold_Price': 'Median_Sold_Price_All'}, inplace=True)
#
median_sold_non_ranch = data[data['Style'] != 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_non_ranch.rename(columns={'Sold_Price': 'Median_Sold_Non_Ranch'}, inplace=True)
#
median_sold_ranch = data[data['Style'] == 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_ranch.rename(columns={'Sold_Price': 'Median_Sold_Ranch'}, inplace=True)

#######
# Merge Median_Sold_Price_All
data = data.merge(median_sold_price_all, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Non_Ranch
data = data.merge(median_sold_non_ranch, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Ranch
data = data.merge(median_sold_ranch, on=['Town', 'Year'], how='left')

houses = data


In [92]:
houses = data


In [93]:
houses.head(2)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09,609500.0,610000.0,520000.0
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11,485000.0,515000.0,440000.0


In [94]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

### Cosolidate House Styles

In [95]:
# Calculate the frequency of each style
style_counts = houses['Style'].value_counts(normalize=True) * 100

# Identify styles that are less than 5%
styles_to_replace = style_counts[style_counts < 5].index
#
# Replace styles that are less than 5% with 'Other'
houses['Style'] = houses['Style'].replace(styles_to_replace, 'Other')

In [96]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

In [97]:
houses.head(2)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09,609500.0,610000.0,520000.0
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11,485000.0,515000.0,440000.0


In [98]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

### Create Monthly Average Sold

In [99]:
# prompt: create new column called 'Call Sold Avg Per Month' and it is the mean of the homes sold each Month_Year

houses['All Sold Avg Per Month'] = houses.groupby('Month_Year')['Sold_Price'].transform('mean')

In [100]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

In [101]:
houses.head(2)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,All Sold Avg Per Month
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09,609500.0,610000.0,520000.0,552586.783088
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11,485000.0,515000.0,440000.0,609019.716578


###
Create Normalized Sold PEr Month

In [102]:
scaler = MinMaxScaler()
houses['Normalized_All Sold Avg Per Month'] = scaler.fit_transform(houses['All Sold Avg Per Month'].values.reshape(-1,1))

In [103]:
houses.head(3)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,All Sold Avg Per Month,Normalized_All Sold Avg Per Month
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09,609500.0,610000.0,520000.0,552586.783088,0.253227
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11,485000.0,515000.0,440000.0,609019.716578,0.356225
2,19 White Spots Lane,Locust Valley,11560,NY,Exp Cape,2427641,650000.0,2.0,1.0,2.5,3.0,Mint,N,Locust Valley,2353.0,White Spots,2011-10-08,NaN,NaN,2011,2011-10,666000.0,770000.0,361000.0,448025.714286,0.062389


In [104]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead',
       'Hicksville', 'East Meadow', 'Freeport', 'Great Neck', 'Uniondale',
       'Roslyns', 'Roosevelt', 'Westbury', 'Valley Stream',
       'Port Washington'], dtype=object)

## Drop duplicate months

In [105]:
# prompt: in "houses" i want to drop duplicate months.  I want one row foe each month

houses = houses.drop_duplicates(subset='Month_Year', keep='last')


In [106]:
# prompt: can i get month_Year in chronological order

houses = houses.sort_values(by='Month_Year', ascending=True)


In [107]:
# prompt: cn i see one example each of teh whatever values are in Town.  unique

houses['Town'].unique()


array(['Port Washington', 'Valley Stream', 'Westbury', 'East Meadow'],
      dtype=object)

## Test plot to see

In [108]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for Product A and Product B
fig.add_trace(go.Scatter(x= houses['Month_Year'], y=houses['Normalized_All Sold Avg Per Month'], mode='lines+markers', name='Houses'))
#fig.add_trace(go.Scatter(x=DF_select_sorted['Month_Year'], y=DF['NormalizedAdjClose'], mode='lines+markers', name='SP 500'))

# Add figure title and labels
fig.update_layout(title='Sales of Product A and B Over Time',
                   xaxis_title='Month_Year',
                   yaxis_title='Sales',
                   template='plotly_dark') # Using a dark theme for the plot

# Show plot
fig.show()

# SP500

## Pre Process SP500

In [109]:
SP500 = pd.read_csv('S&P500_yahoo.csv')

## Create AdjClose Mean

In [110]:
SP500.head(2)

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Jan 23, 2024","4,856.80","4,866.48","4,844.37","4,864.60","4,864.60","3,912,800,000"
1,"Jan 22, 2024","4,853.42","4,868.41","4,844.05","4,850.43","4,850.43","4,297,610,000"


In [111]:
import pandas as pd

# Assuming your SP500 DataFrame is already defined and loaded with the 'date' column
# First, make sure the 'date' column is in datetime format
SP500['Date'] = pd.to_datetime(SP500['Date'])

# Now, create the 'Month_Year' column formatted as "Month YYYY"
SP500['Month_Year'] = SP500['Date'].dt.strftime('%b %Y')

# Show the first few rows to verify
SP500.head()


,Date,Open,High,Low,Close*,Adj Close**,Volume,Month_Year
0,2024-01-23,"4,856.80","4,866.48","4,844.37","4,864.60","4,864.60","3,912,800,000",Jan 2024
1,2024-01-22,"4,853.42","4,868.41","4,844.05","4,850.43","4,850.43","4,297,610,000",Jan 2024
2,2024-01-19,"4,796.28","4,842.07","4,785.87","4,839.81","4,839.81","4,287,200,000",Jan 2024
3,2024-01-18,"4,760.10","4,785.79","4,740.57","4,780.94","4,780.94","4,019,000,000",Jan 2024
4,2024-01-17,"4,739.13","4,744.23","4,714.82","4,739.21","4,739.21","3,928,600,000",Jan 2024


In [112]:
# Example cleanup: Remove commas and convert to numeric
SP500['Adj Close**'] = SP500['Adj Close**'].str.replace(',', '').astype(float)


In [113]:
import pandas as pd

# Assuming your SP500 DataFrame is already defined
# Convert 'Adj Close**' to numeric, forcing non-numeric values to NaN (errors='coerce')
SP500['Adj Close**'] = pd.to_numeric(SP500['Adj Close**'], errors='coerce')

# Group by 'Month_Year' and calculate the mean of 'Adj Close**'
mean_adj_close_by_month_year = SP500.groupby('Month_Year')['Adj Close**'].mean()

# Reset index if you want the result as a DataFrame
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year.reset_index()

# Show the result
#print(mean_adj_close_by_month_year_df)
mean_adj_close_by_month_year_df.head(5)

,Month_Year,Adj Close**
0,Apr 2006,1302.165263
1,Apr 2007,1463.640500
2,Apr 2008,1370.469091
3,Apr 2009,848.151905
4,Apr 2010,1197.316190


## Normalizing the monthly adj close price mean

In [114]:
# Assume mean_adj_close_by_month_year_df is your DataFrame and 'Adj Close**' is the column to be normalized

# Calculating Min and Max values
min_value = mean_adj_close_by_month_year_df['Adj Close**'].min()
max_value = mean_adj_close_by_month_year_df['Adj Close**'].max()

# Applying the normalization formula
mean_adj_close_by_month_year_df['NormalizedAdjClose'] = (mean_adj_close_by_month_year_df['Adj Close**'] - min_value) / (max_value - min_value)

# Display the first few rows to verify the new column
mean_adj_close_by_month_year_df.head(3)

,Month_Year,Adj Close**,NormalizedAdjClose
0,Apr 2006,1302.165263,0.135840
1,Apr 2007,1463.640500,0.176085
2,Apr 2008,1370.469091,0.152864


## Makes sure SP500 is in chrono order

In [115]:
# prompt: lets make sure that the df "mean_adj_close_by_month_year_df" has both only one row per month and is in chronological order

# Sort the DataFrame by 'Month_Year'
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.sort_values(by='Month_Year')

# Drop duplicate rows based on 'Month_Year'
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.drop_duplicates(subset='Month_Year', keep='last')

# Reset the index to ensure consecutive indexing
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.reset_index(drop=True)

# Now, mean_adj_close_by_month_year_df has one row per month in chronological order


## Now lets have a test plot

In [116]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for Product A and Product B
fig.add_trace(go.Scatter(x= mean_adj_close_by_month_year_df['Month_Year'], y= mean_adj_close_by_month_year_df['NormalizedAdjClose'], mode='lines+markers', name='Houses'))
#fig.add_trace(go.Scatter(x=DF_select_sorted['Month_Year'], y=DF['NormalizedAdjClose'], mode='lines+markers', name='SP 500'))

# Add figure title and labels
fig.update_layout(title='Sales of Product A and B Over Time',
                   xaxis_title='Month_Year',
                   yaxis_title='Sales',
                   template='plotly_dark') # Using a dark theme for the plot

# Show plot
fig.show()

# Now Merge the houses and mean_adj_close_by_month_year_df on Month Year

In [117]:
import pandas as pd

# Convert 'Month_Year' to datetime
mean_adj_close_by_month_year_df['Month_Year'] = pd.to_datetime(mean_adj_close_by_month_year_df['Month_Year'], format='%b %Y')

# Format 'Month_Year' to "YYYY-MM"
mean_adj_close_by_month_year_df['Month_Year'] = mean_adj_close_by_month_year_df['Month_Year'].dt.strftime('%Y-%m')

# Now, both DataFrames should have the 'Month_Year' column formatted as "YYYY-MM", and you can merge them.
DF = pd.merge(houses, mean_adj_close_by_month_year_df, on='Month_Year', how='inner')

# Display the first few rows of the merged DataFrame
DF.head(3)


,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,All Sold Avg Per Month,Normalized_All Sold Avg Per Month,Adj Close**,NormalizedAdjClose
0,54 Farmview Road,Port Washington,11050,NY,Colonial,1779948,1500000.0,3.0,1.0,3.50,5.0,Mint,NaN,Port Washington,NaN,Farmview,2006-01-31,NaN,NaN,2006,2006-01,735000.0,740000.0,649000.0,559653.061224,0.266124,1275.726667,0.129251
1,44 Murray Avenue,Port Washington,11050,NY,Other,1780371,1100000.0,3.0,2.0,3.55,6.0,Mint,NaN,Port Washington,NaN,Murray,2006-02-28,NaN,NaN,2006,2006-02,735000.0,740000.0,649000.0,522184.166667,0.197738,1276.645263,0.129480
2,5 Lewis Lane,Port Washington,11050,NY,Ranch,1796826,649000.0,1.0,1.0,1.50,3.0,Excellent,NaN,Port Washington,NaN,Lewis,2006-03-17,NaN,NaN,2006,2006-03,735000.0,740000.0,649000.0,541780.848214,0.233505,1293.734783,0.133739


In [118]:
DF.head(3)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,All Sold Avg Per Month,Normalized_All Sold Avg Per Month,Adj Close**,NormalizedAdjClose
0,54 Farmview Road,Port Washington,11050,NY,Colonial,1779948,1500000.0,3.0,1.0,3.50,5.0,Mint,NaN,Port Washington,NaN,Farmview,2006-01-31,NaN,NaN,2006,2006-01,735000.0,740000.0,649000.0,559653.061224,0.266124,1275.726667,0.129251
1,44 Murray Avenue,Port Washington,11050,NY,Other,1780371,1100000.0,3.0,2.0,3.55,6.0,Mint,NaN,Port Washington,NaN,Murray,2006-02-28,NaN,NaN,2006,2006-02,735000.0,740000.0,649000.0,522184.166667,0.197738,1276.645263,0.129480
2,5 Lewis Lane,Port Washington,11050,NY,Ranch,1796826,649000.0,1.0,1.0,1.50,3.0,Excellent,NaN,Port Washington,NaN,Lewis,2006-03-17,NaN,NaN,2006,2006-03,735000.0,740000.0,649000.0,541780.848214,0.233505,1293.734783,0.133739


### Lets select a fw cols

# Plot

### Lets try a line chart of house and SP monthly means

In [119]:
#multiple_columns = df[['ColumnName1', 'ColumnName2']]
DF_select = DF[['Month_Year', 'Normalized_All Sold Avg Per Month' , 'NormalizedAdjClose']]


In [ ]:
DF_select.head(30)

In [ ]:
DF_select

## Getthe Month Year in Chron order

In [ ]:
import pandas as pd

# Assuming DF_select is your DataFrame
# Convert 'Month_Year' to datetime format
DF_select['Month_Year'] = pd.to_datetime(DF_select['Month_Year'], format='%Y-%m')

# Now, sort by 'Month_Year' in chronological order
DF_select_sorted = DF_select.sort_values(by='Month_Year')

# Display the sorted DataFrame
print(DF_select_sorted)


## We only wasnt one of each date

In [123]:
DF_select_sorted.head(5)

,Month_Year,Normalized_All Sold Avg Per Month,NormalizedAdjClose
0,2006-01-01,0.266124,0.129251
1,2006-02-01,0.197738,0.129480
2,2006-03-01,0.233505,0.133739
3,2006-04-01,0.334784,0.135840
4,2006-05-01,0.271757,0.132812


In [124]:
# prompt: my data DF_select_sorted has multiplesof each date.  I would only like the first of each date

DF_select_sorted.drop_duplicates(subset='Month_Year', keep='first', inplace=True)


In [ ]:
DF_select_sorted.head(5)

In [126]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for Product A and Product B
fig.add_trace(go.Scatter(x= DF_select_sorted['Month_Year'], y=DF['Normalized_All Sold Avg Per Month'], mode='lines+markers', name='Houses'))
fig.add_trace(go.Scatter(x=DF_select_sorted['Month_Year'], y=DF['NormalizedAdjClose'], mode='lines+markers', name='SP 500'))

# Add figure title and labels
fig.update_layout(title='Sales of Product A and B Over Time',
                   xaxis_title='Month_Year',
                   yaxis_title='Sales',
                   template='plotly_dark') # Using a dark theme for the plot

# Show plot
fig.show()